# **Analiza danych ankietowych - Sprawozdanie I**

## *Autorzy: Julia Krempińska, Filip Miśkiewicz*

## **Biblioteki**

In [ ]:
import numpy as np
import scipy.stats as st
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline 

## **Przygotowanie danych**

In [20]:
# Wczytanie danych z pliku ankieta.csv
df = pd.read_csv("ankieta.csv", sep=";")

In [21]:
WIEK_KAT = np.zeros(len(df["WIEK"]))
for i in range(len(df["WIEK"])): #sprawdzanie warunków
    if 35 < df["WIEK"][i] <= 45:
        WIEK_KAT[i] = 1
    if 45 < df["WIEK"][i] <= 55:
        WIEK_KAT[i] = 2
    if df["WIEK"][i] > 55:
        WIEK_KAT[i] = 3
df["WIEK_KAT"] = WIEK_KAT
df.tail() #0 jeśli < 35

,DZIAŁ,STAŻ,CZY_KIER,PYT_1,PYT_2,PYT_3,PŁEĆ,WIEK,WIEK_KAT
195,HR,2,Nie,1,2,2,M,42,1.0
196,HR,2,Nie,1,-1,-1,K,35,0.0
197,HR,2,Nie,-1,-2,-2,K,39,1.0
198,HR,2,Nie,1,2,1,K,48,2.0
199,HR,2,Nie,1,2,2,K,42,1.0


## **Część I**

### **Zadanie 1**

Zadanie polega na oszacowaniu przedziałów ufności dla prawdopodobieństw odpowiedzi pracowników w pięciostopniowej skali oceny szkolenia.

Zakładamy, że liczność próby $n = 200$ jest wystarczająco duża, aby można było zastosować normalne przybliżenie przedziału ufności dla estymatora częstości $\hat{p}_i$:

$$
\hat{p}_i \pm z_{\alpha/2} \cdot \sqrt{\frac{\hat{p}_i (1 - \hat{p}_i)}{n}}
$$

gdzie:
- $\hat{p}_i = \frac{x_i}{n}$ – estymator prawdopodobieństwa,
- $z_{\alpha/2} = 1{.}96$ dla poziomu ufności 0.95,
- $n = 200$ – liczność próby.

In [22]:
n = 200

counts = np.array([14, 17, 40, 100, 29])
categories = [
    "Bardzo niezadowolony", 
    "Niezadowolony", 
    "Bez zdania", 
    "Zadowolony", 
    "Bardzo zadowolony"
]

# Estymatory p̂
p_hat = counts / n

# Parametry dla CI
alpha = 0.05
z = st.norm.ppf(1 - alpha / 2)

# Liczenie błędu standardowego bezpiecznie
standard_error = np.sqrt(np.clip(p_hat * (1 - p_hat) / n, 0, None))

# Przedziały ufności
ci_lower = p_hat - z * standard_error
ci_upper = p_hat + z * standard_error

# Ograniczenie do [0, 1]
ci_lower = np.clip(ci_lower, 0, 1)
ci_upper = np.clip(ci_upper, 0, 1)

# Tabela wyników
df_res = pd.DataFrame({
    "Kategoria": categories,
    "Liczba odpowiedzi": counts,
    "Estymator p̂": p_hat,
    "95% CI dolny": ci_lower,
    "95% CI górny": ci_upper
})

df_res = df_res.round(4)

display(df_res)


,Kategoria,Liczba odpowiedzi,Estymator p̂,95% CI dolny,95% CI górny
0,Bardzo niezadowolony,14,0.070,0.0346,0.1054
1,Niezadowolony,17,0.085,0.0463,0.1237
2,Bez zdania,40,0.200,0.1446,0.2554
3,Zadowolony,100,0.500,0.4307,0.5693
4,Bardzo zadowolony,29,0.145,0.0962,0.1938


## Wnioski

- Przedziały są najszersze dla kategorii o małej liczebności (np. "bardzo niezadowolony").
- Najwęższy przedział dotyczy kategorii "zadowolony", co wynika z dużej liczby odpowiedzi.
- Przedziały te pozwalają oszacować niepewność związania z odpowiedziami i mogą służyć do porównań między grupami.

### **Zadanie 2**

Zadanie polega na implementacji dwóch testów służących do weryfikacji hipotezy zerowej $H_0: \mathbf{p} = \mathbf{p}_0$ dla danych z rozkładu wielomianowego:

- **Test chi-kwadrat Pearsona** – klasyczny test zgodności,
- **Test największej wiarygodności (G-test)** – oparty na log-likelihood ratio.

Te testy porównują obserwowane częstości z oczekiwanymi na podstawie hipotezy $H_0$.

#### Wzory statystyk:

Chi-kwadrat Pearsona:
$$
\chi^2 = \sum_{i=1}^{k} \frac{(x_i - n p_{0i})^2}{n p_{0i}}
$$

G-test (log-likelihood ratio):
$$
G^2 = 2 \sum_{i=1}^{k} x_i \ln \left( \frac{x_i}{n p_{0i}} \right)
$$

Jeśli $x_i = 0$, to składnik $x_i \ln\left(\frac{x_i}{(n p_{0i})}\right)$ definiujemy jako $0$, aby uniknąć problemu z logarytmem z zera – to zgodne z regułą: $\lim_{x \to 0^+} x \ln x = 0$.

In [23]:
def chi_squared_test(x, p0, alpha=0.05):
    """
    Test chi-kwadrat Pearsona i największej wiarygodności dla hipotezy H0: p = p0
    
    Parametry:
    x     - wektor zliczeń (np. [14, 17, 40, 100, 29])
    p0    - wektor p0 z hipotezy zerowej (np. [0.1, 0.1, 0.2, 0.4, 0.2])
    alpha - poziom istotności (np. 0.05)
    
    Zwraca:
    Słownik z wynikami testów: statystyki, p-value, decyzja o odrzuceniu H0
    """
    x = np.array(x)
    p0 = np.array(p0)
    n = np.sum(x)
    k = len(x)
    
    # Oczekiwane wartości
    expected = n * p0

    # Statystyka testu Pearsona
    with np.errstate(divide='ignore', invalid='ignore'):
        T_Pearson = np.nansum((x - expected)**2 / expected)

    # Statystyka testu największej wiarygodności (log-likelihood ratio)
    ratio = np.where(x == 0, 1, x / expected)
    T_LR = 2 * np.nansum(np.where(x == 0, 0, x * np.log(ratio)))

    # Stopnie swobody
    df = k - 1

    # Wartości krytyczne
    critical_value = st.chi2.ppf(1 - alpha, df)

    # P-value
    pval_pearson = 1 - st.chi2.cdf(T_Pearson, df)
    pval_lr = 1 - st.chi2.cdf(T_LR, df)

    # Decyzja
    decision_pearson = "Odrzucamy H0" if pval_pearson < alpha else "Brak podstaw do odrzucenia H0"
    decision_lr = "Odrzucamy H0" if pval_lr < alpha else "Brak podstaw do odrzucenia H0"

    return {
        "Test Pearsona": {
            "Statystyka": T_Pearson,
            "p-value": pval_pearson,
            "Wartość krytyczna": critical_value,
            "Decyzja": decision_pearson
        },
        "Test największej wiarygodności": {
            "Statystyka": T_LR,
            "p-value": pval_lr,
            "Wartość krytyczna": critical_value,
            "Decyzja": decision_lr
        },
        "Stopnie swobody": df,
        "Poziom istotności": alpha
    }


### **Zadanie 3**

Celem jest weryfikacja, czy rozkład odpowiedzi na pytanie PYT_1 w grupie pracowników Działu Produktowego jest **równomierny** – tzn. każda odpowiedź jest równie prawdopodobna ($p_0 = [0.2, 0.2, 0.2, 0.2, 0.2]$).

### Hipotezy:

- $H_0$: rozkład odpowiedzi jest równomierny,
- $H_1$: rozkład odpowiedzi nie jest równomierny.

Weryfikacja następuje przy użyciu testu $\chi^2$ Pearsona z funkcji z zadania 2.

In [24]:
dfPD = df[df["DZIAŁ"] == "PD"]
counts = (dfPD['PYT_1'].value_counts())
print(counts)
     
p0 = np.full(len(counts), 1 / len(counts))  
print(p0)

T_Pearson= chi_squared_test(counts, p0, alpha=0.05)

display("Statystyka chi-kwadrat Pearsona:", T_Pearson)

PYT_1
 1    51
 0    17
 2    11
-1    10
-2     9
Name: count, dtype: int64
[0.2 0.2 0.2 0.2 0.2]


'Statystyka chi-kwadrat Pearsona:'

{'Test Pearsona': {'Statystyka': 64.85714285714286,
  'p-value': 2.757793993168889e-13,
  'Wartość krytyczna': 9.487729036781154,
  'Decyzja': 'Odrzucamy H0'},
 'Test największej wiarygodności': {'Statystyka': 52.527113506941156,
  'p-value': 1.0701994845874196e-10,
  'Wartość krytyczna': 9.487729036781154,
  'Decyzja': 'Odrzucamy H0'},
 'Stopnie swobody': 4,
 'Poziom istotności': 0.05}

## Wnioski

- Wartość p-value jest mniejsza niż 0.05, a zatem odrzucamy hipotezę $H_0$ i rozkład nie jest równomierny.
- Test pozwala sprawdzić, czy odpowiedzi są losowe, czy np. preferowane są bardziej pozytywne/negatywne oceny.


## **Część II**

### **Zadanie 4**

#### Test Fishera (dla tabel 2×2)

Test Fishera to dokładny test statystyczny stosowany do analizy zależności pomiędzy dwiema zmiennymi jakościowymi w tabeli 2×2, szczególnie gdy liczebności są małe. Zakłada on ustalone marginesy (wiersze i kolumny).

- **Hipoteza zerowa (H₀):** zmienne są niezależne.
- **Hipoteza alternatywna (H₁):** zmienne są zależne.

Statystyka testu opiera się na rozkładzie hipergeometrycznym:

$$
P = \frac{\binom{a+b}{a} \binom{c+d}{c}}{\binom{n}{a+c}}
$$

gdzie:
- \( a, b, c, d \) to liczby w tabeli 2×2,
- \( n = a + b + c + d \).

#### Test Freemana-Haltona

Rozszerzenie testu Fishera na tabele większe niż 2×2 (RxC). Oblicza dokładne p-value przy założeniu niezależności. Ze względu na złożoność, często stosuje się przybliżenia metodą Monte Carlo.

Do wykonania testu Fishera użyjemy funkcji z biblioteki *scipy.stats*, a do testu Freemana-Haltona samodzielnie napisanej funkcji.

In [25]:
def fisher_monte_carlo_from_table(data1, data2, B=10000, random_state=None):
    data_frame = pd.crosstab(df[data1], df[data2])

    np.random.seed(random_state)

    # Oryginalna statystyka chi-kwadrat
    chi2_obs, _, _, _ = st.chi2_contingency(data_frame, correction=False)

    # Rekonstrukcja "płaskiego" dataframe do permutacji
    expanded_df = df[[data1, data2]]
    
    shuffled = np.random.permutation(expanded_df[data2])
    simulated_table = pd.crosstab(expanded_df[data1], shuffled)

    # Symulacje
    count = 0
    for _ in range(B):
        shuffled = np.random.permutation(expanded_df[data2])
        simulated_table = pd.crosstab(expanded_df[data1], shuffled)
        
        # Dopasuj do kształtu oryginału (w razie braków)
        simulated_table = simulated_table.reindex(index=data_frame.index, columns=data_frame.columns, fill_value=0)
        
        chi2_sim, _, _, _ = st.chi2_contingency(simulated_table, correction=False)
        
        if chi2_sim >= chi2_obs:
            count += 1

    p_value = count / B
    return chi2_obs, p_value

### **Zadanie 5**

Celem jest sprawdzenie, czy zmienne **PŁEĆ** i **CZY_KIER** są niezależne.

- **H₀:** PŁEĆ i CZY_KIER są niezależne.
- **H₁:** Pomiędzy PŁEĆ a CZY_KIER istnieje zależność.

In [26]:
df2 = pd.crosstab(df['PŁEĆ'], df['CZY_KIER'])
display(df2)

odds_ratio, p_value = st.fisher_exact(df2, alternative='two-sided')
    
decision = "Odrzucamy H₀ – zmienne są zależne" if p_value < 0.05 else "Brak podstaw do odrzucenia H₀ - zmienne są niezależne"

display({
    "p-value": p_value,
    "Decyzja": decision
})

CZY_KIER,Nie,Tak
PŁEĆ,,
K,63,8
M,110,19


{'p-value': 0.6659028889666552,
 'Decyzja': 'Brak podstaw do odrzucenia H₀ - zmienne są niezależne'}

## Wnioski

- p_value jest większe niż 0.05, a zatem nie ma podstaw do odrzucenia $H_0$ i możemy założyć, że zmienne są niezależne

### **Zadanie 6**

Celem jest zbadanie, czy wybrane zmienne jakościowe są statystycznie zależne, za pomocą testu **Freemana-Haltona** (rozszerzenia testu Fishera dla tabel większych niż 2×2), na poziomie istotności α = 0.05.

In [27]:
# Funkcja do generowania wniosków na podstawie p-value
def check_pvalue(alpha=0.05):
    if p_value < alpha:
        print("\nWniosek: Odrzucamy hipotezę zerową (jest zależność).")
    else:
        print("\nWniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).")

In [28]:
# Poziom istotności
alpha=0.05

a) zajmowanie stanowiska kierowniczego nie zależy od wieku 

In [29]:
# Przykład użycia:
chi2_obs, p_value = fisher_monte_carlo_from_table('WIEK_KAT', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 1.0803155311825832
P-wartość (Monte Carlo) = 0.794

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


b) zajmowanie stanowiska kierowniczego nie zależy od stażu pracy

In [30]:
chi2_obs, p_value = fisher_monte_carlo_from_table('STAŻ', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 23.179908824020515
P-wartość (Monte Carlo) = 0.0

Wniosek: Odrzucamy hipotezę zerową (jest zależność).


c) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb
w pierwszym badanym okresie nie zależy od zajmowanego stanowiska

In [31]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 13.11369172640849
P-wartość (Monte Carlo) = 0.021

Wniosek: Odrzucamy hipotezę zerową (jest zależność).


d) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w
pierwszym badanym okresie nie zależy od stażu 

In [32]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'STAŻ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 26.1193138645
P-wartość (Monte Carlo) = 0.001

Wniosek: Odrzucamy hipotezę zerową (jest zależność).


e) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w
pierwszym badanym okresie nie zależy od płci

In [33]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'PŁEĆ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 2.2700823224897917
P-wartość (Monte Carlo) = 0.555

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


f) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w
pierwszym badanym okresie nie zależy od wieku

In [34]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'WIEK_KAT', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 9.845826003518312
P-wartość (Monte Carlo) = 0.35

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [35]:
# Modyfikacja kolumny PYT_2
CZY_ZADOW = np.zeros(len(df["PYT_2"]))
for i in range(len(df["PYT_2"])):
    CZY_ZADOW[i] = np.sign(df["PYT_2"][i]) #pozytywna odpowiedź > 0, negatywna < 0
df["CZY_ZADOW"] = CZY_ZADOW
df.tail()

,DZIAŁ,STAŻ,CZY_KIER,PYT_1,PYT_2,PYT_3,PŁEĆ,WIEK,WIEK_KAT,CZY_ZADOW
195,HR,2,Nie,1,2,2,M,42,1.0,1.0
196,HR,2,Nie,1,-1,-1,K,35,0.0,-1.0
197,HR,2,Nie,-1,-2,-2,K,39,1.0,-1.0
198,HR,2,Nie,1,2,1,K,48,2.0,1.0
199,HR,2,Nie,1,2,2,K,42,1.0,1.0


In [36]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 0.08183603554685095
P-wartość (Monte Carlo) = 0.858

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [37]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'STAŻ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 1.7773575924124572
P-wartość (Monte Carlo) = 0.435

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [38]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'PŁEĆ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 0.23290747005849552
P-wartość (Monte Carlo) = 0.662

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [39]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'WIEK_KAT', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 3.5157335639069873
P-wartość (Monte Carlo) = 0.321

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).
